### Listen von Listen

In [ ]:
empty_row = [' '] * 4
board = [empty_row for _ in range(4)]
empty_row[0] = '*'
board

In [ ]:
empty_row = [' '] * 4
board = [empty_row.copy() for _ in range(4)]
empty_row[0] = '*'
board

In [ ]:
board = [[' '] * 4 for _ in range(4)]
board

In [ ]:
def new_board(xmax=8, ymax=8):
    return [[' '] * xmax for _ in range(ymax)]


def set_item(board, pos, val):
    x, y = pos
    board[y][x] = val


def get_item(board, pos, val):
    x, y = pos
    val = board[y][x]
    return val


def is_on_board(pos, xmax=8, ymax=8):
    x, y = pos
    return 0 <= x < xmax and 0 <= y < ymax


def get_board_size(board):
    if not board:
        return (0, 0)
    y = len(board)
    if not board[0]:
        return (0, y)
    return (len(board[0]), y)

In [ ]:
def xy2chess(pos, cols='ABCDEFGH', rows='12345678'):
    x, y = pos
    return cols[x]+rows[y]


def chess2xy(pos, cols='ABCDEFGH', rows='12345678'):
    c, r = pos
    assert c in cols and r in rows, f'{pos} ist kein gueltiges Feld!'
    return ord(c)-ord(cols[0]), ord(r)-ord(rows[0])


def show(xys):
    print([xy2chess(xy) for xy in xys])

In [ ]:
show([(0, 0), (1, 2)])

In [ ]:
xy2chess((1,2))

In [ ]:
chess2xy('B3')

In [ ]:
CMDS = {'u': (0, 1),
        'd': (0, -1),
        'r': (1, 0),
        'l': (-1, 0),
        }

KING = ('l', 'ul', 'u', 'ur', 'r', 'dr', 'd', 'dl')
KNIGHT = ('ull', 'uul', 'uur', 'urr',  'drr',  'ddr', 'ddl', 'dll')

In [ ]:
def move(pos, cmds, steps=1):
    x, y = pos
    for c in cmds:
        if c not in CMDS:
            raise ValueError(f'{c} is not a valid command!')
        dx, dy = CMDS[c]
        x = x + dx*steps
        y = y + dy*steps
    return (x, y)

In [ ]:
move((2, 3), 'ur', 2)

In [ ]:
def knight_moves(pos, xmax=8, ymax=8):
    xys = []
    for cmd in KNIGHT:
        p = move(pos, cmd)
        if is_on_board(p, xmax, ymax):
            xys.append(p)
    return xys

In [ ]:
field = 'E4'
pos = chess2xy(field)
xys = knight_moves(pos)
[xy2chess(xy) for xy in xys]

In [ ]:
def king_moves(pos, xmax=8, ymax=8):
    xys = []
    for cmd in KING:
        p = move(pos, cmd)
        if is_on_board(p, xmax, ymax):
            xys.append(p)
    return xys

In [ ]:
field = 'E4'
pos = chess2xy(field)
xys = king_moves(pos)
[xy2chess(xy) for xy in xys]

In [ ]:
def queen_moves(pos, xmax=8, ymax=8):
    xys = []
    for cmd in KING:
        for i in range(1, 8):
            xy = move(pos, cmd, steps=i)
            if is_on_board(xy, xmax, ymax):
                xys.append(xy)
    return xys

In [ ]:
field = 'D4'
pos = chess2xy(field)
xys = queen_moves(pos)
', '.join([xy2chess(xy) for xy in xys])

In [ ]:
relations = {'x': 'gleiche Reihe',
             'y': 'gleiche Kolonne',
             'D': 'gleiche Hautdiagonale',
             'd': 'gleiche Nebendiagonale',
             '=': 'identisch',
             None: 'keine Beziehung',
             }


def get_relative_position(pos1, pos2):
    dx = pos1[0] - pos2[0]
    dy = pos1[1] - pos2[1]

    r = None
    if pos1 == pos2:
        r = '='
    elif dx == 0:
        r = 'x'
    elif dy == 0:
        r = 'y'
    elif dx == dy:
        r = 'D'
    elif dx == -dy:
        r = 'd'

    return r


def is_save(queens, pos):
    for queen in queens:
        if get_relative_position(queen, pos) is not None:
            return False
    return True

In [ ]:
field = 'D4'
pos = chess2xy(field)
xys = queen_moves(pos)
for xy in xys:
    r = get_relative_position(pos, xy)
    print(f'{field} und {xy2chess(xy)}: {relations[r]}')

In [ ]:
queens = []
N = 8
y = 0

while len(queens) < N:
    x = len(queens)
    if y < N and is_save(queens, (x, y)):
        queens.append((x, y))
        y = 0
    elif y < N-1:
        y = y + 1
    else:
        print([xy2chess(xy) for xy in queens])
        y = queens[-1][1] + 1
        queens.pop()

queens

In [ ]:
start = (0, 0)
options = knight_moves(start, N, N)
[opt for opt in options if opt in ((1, 2), (2, 1))]

In [114]:
start = (0, 0)
knight_tour = []
seen = set()
N = 8
stack = [(0, start)]


steps = 0
while len(knight_tour) < N**2 and steps < 1_000_000:
    steps = steps + 1
    i, xy = stack.pop()
    for i in range(len(knight_tour)-i):
        seen.remove(knight_tour.pop())
    knight_tour.append(xy)
    seen.add(xy)
    options = knight_moves(xy, N, N)
    # works (else no path is found on 8x8 board)
    options.sort(key=lambda x: len(set(knight_moves(x, N, N)) - seen), reverse=True)
    stack.extend([(len(knight_tour), xy) for xy in options if xy not in seen])

len(knight_tour), steps

(64, 64)

In [116]:
show(knight_tour)

['A1', 'C2', 'E1', 'G2', 'H4', 'G6', 'H8', 'F7', 'H6', 'G8', 'E7', 'C8', 'A7', 'B5', 'A3', 'B1', 'D2', 'F1', 'H2', 'G4', 'E3', 'D1', 'B2', 'A4', 'C3', 'A2', 'C1', 'B3', 'A5', 'C4', 'B6', 'A8', 'C7', 'A6', 'B8', 'C6', 'B4', 'D5', 'F4', 'D3', 'E5', 'F3', 'G1', 'H3', 'F2', 'H1', 'G3', 'E2', 'D4', 'F5', 'G7', 'H5', 'F6', 'E8', 'D6', 'E4', 'G5', 'H7', 'F8', 'D7', 'C5', 'E6', 'D8', 'B7']


In [118]:
N = 8
queens = []
start = (0, 0)
stack = [(0, start)]

n = len(queens)
while n < N:
    i, xy = stack.pop()
    for i in range(n-i):
        queens.pop()
    queens.append(xy)
    n = len(queens)
    options = [(n, i) for i in range(N) if is_save(queens, (n, i))]
    stack.extend([(n, xy) for xy in options])

show(queens), steps

['A1', 'B7', 'C5', 'D8', 'E2', 'F4', 'G6', 'H3']


(None, 44)

In [123]:
def is_save_(queens, row):
    n = len(queens)
    return is_save([(i, q) for i, q in enumerate(queens)], (n, row))

In [ ]:
queens = [0, 3]
[row for row in range(N) if is_save_(queens, row)]

In [126]:
def is_save1(queens, row, col):
    for i in range(col):
        save = queens[i] != row and abs(i-col) != abs(queens[i]-row)
        if not save:
            return False
    return True

In [127]:
queens = [0, 3]
[row for row in range(N) if is_save1(queens, row, 2)]

[1, 5, 6, 7]

In [125]:
N = 8
queens = [0] * N
start = 0
stack = [(0, start)]

n = 0
while n < N:
    i, row = stack.pop()
    queens[i] = row
    n = i + 1
    options = [row for row in range(N) if is_save_(queens[:n], row)]
    stack.extend([(n, row) for row in options])

queens

[0, 6, 4, 7, 1, 3, 5, 2]

In [128]:
N = 8
queens = [0] * N
start = 0
stack = [(0, start)]

n = 0
while n < N:
    i, row = stack.pop()
    queens[i] = row
    n = i + 1
    options = [row for row in range(N) if is_save1(queens, row, n)]
    stack.extend([(n, row) for row in options])

queens

[0, 6, 4, 7, 1, 3, 5, 2]